# 뉴스데이터 연관키워드 분석 (2020년 07월 01일 ~ 07월 05일) / 형태소분석 적용

In [3]:
%matplotlib inline

In [2]:
try:
    import pandas as pd
except:
    !pip install pandas
    !pip install numpy==1.19.0
    !pip install matplotlib
    import pandas as pd
    

import numpy as np
import matplotlib.pyplot as plt

import warnings

In [3]:
warnings.filterwarnings("ignore")

## 2. 키워드 추출

#### [2-1) 텍스트 데이터 전처리]

In [48]:
import re

# 텍스트 정제 함수 : 할글 이외의 문자는 전부 제거
def text_cleaning(text):
#     hangul = re.compile('[^ㄱ-ㅎ|가-힣|a-z|A-Z|0-9|\*]') #한글의 정규표현식을 나타냅니다.
    hangul = re.compile('[^ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]') #한글의 정규표현식을 나타냅니다.
    result = hangul.sub(' ', text)
    result = re.sub(' +', ' ', result)
    return result



#### [2-2) KoNLPy를 이용한 키워드 추출 (형태소 분석기)]
#### 한국어 약식 불용어 사전 사용 (http://ranks.nl/stopwords/korean)

In [49]:
# csv파일 불러오기
df = pd.read_csv('news_data_ori_0701_0705.csv')
df.head()

print(df)

df['ko_text'] = df['본문'].apply(lambda x: text_cleaning(x))
df.head()


#konlpy 문서 : https://konlpy-ko.readthedocs.io/ko/v0.4.3/api/konlpy.tag/
#형태소 분석 태그정리 : https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0
#사전 튜닝 : https://cromboltz.tistory.com/18

# from konlpy.tag import Okt
from eunjeon import Mecab
from collections import Counter
from IPython.display import clear_output


korean_stopwords_path = './korean_stopwords.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

nouns_cnt = 1
count_data = []

def get_nouns(x):
    global nouns_cnt, count_data
    
#     nouns_tagger = Okt()
    nouns_tagger=Mecab(dicpath='D:/DEV/python/workspace_python/python1/0.prototype/lib/mecab/mecab-ko-dic')
    nouns = nouns_tagger.nouns(x)
    count_data.extend([y for y in nouns])
#     count_data.extend([z for y in nouns for z in y])

    #한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]

    #불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]

    clear_output(wait=True)
    print(f'형태소,불용어 제거중 .. {round((nouns_cnt / df.shape[0])*100,3)} %')
    nouns_cnt+=1
    
    return nouns



                                                      본문  \
0       인천해양경찰서(서장 신동삼)는 도서 지역 치안 강화를 위해 인천 중부경찰서와 지난...   
1       작년말 대비 두 배 가까이 늘어 지난달 26일 하룻새 4조 급증 SK바이오팜 청약...   
2       ［중부매일 서인석 기자］ 이차영 괴산군수가 1일 열린 직원조회에서 민선 7기 2년...   
3       라임자산운용(라임)의 1조6000억원 펀드 환매 중단 사건을 수사 중인 검찰이 1...   
4       시모네타 소마루가 스위스 대통령 [EPA=연합뉴스 자료사진] . (제네바=연합뉴스...   
...                                                  ...   
21189   월요일인 6일은 전국이 가끔 구름이 많고 더운 가운데 소나기가 오는 곳이 있겠다....   
21190   한국여자프로골프(KLPGA) 투어 맥콜-용평리조트 오픈 with SBS골프에서 김...   
21191   [앵커] 추미애 법무부 장관의 '검·언 유착 의혹' 사건 지휘권 행사와 관련해, ...   
21192   세계보건기구(WHO)가 말라리아 치료제 클로로퀸과 HIV(인간면역결핍바이러스) 치...   
21193   '검언유착' 의혹 수사와 관련한 추미애 법무부 장관의 수사지휘에 윤석열 검찰총장이...   

                                                     형태소  
0       인천해양경찰/NNG 서/JKB (/SS 서장/NNG 신동삼/NNG )/SS 는/J...  
1       작년/NNG 말/NNB 대비/NNG 두/MM 배/NNG 가까이/MAG 늘/VV 어...  
2       ［/SL 중부매/NNG 이/VCP ㄹ/ETM 서인석/NNP 기자/NNG ］/SL ...  
3       라임자산운/NNG 용/XSN (/SS 라임/NNG )/SS 의/

In [50]:
# df = df.iloc[:1000,:]


import time
start = time.time()  # 시작 시간 저장

nouns_cnt = 1
# df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
# df['nouns'] = df['본문'].apply(lambda x: get_nouns(x))
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


print(df.shape)
df.head()

형태소,불용어 제거중 .. 100.0 %
time : 190.01002597808838
(21194, 4)


,본문,형태소,ko_text,nouns
0,인천해양경찰서(서장 신동삼)는 도서 지역 치안 강화를 위해 인천 중부경찰서와 지난...,인천해양경찰/NNG 서/JKB (/SS 서장/NNG 신동삼/NNG )/SS 는/J...,인천해양경찰서 서장 신동삼 는 도서 지역 치안 강화를 위해 인천 중부경찰서와 지난...,"[인천, 해양경찰, 서장, 신동, 도서, 지역, 치안, 강화, 인천, 중부, 경찰서..."
1,작년말 대비 두 배 가까이 늘어 지난달 26일 하룻새 4조 급증 SK바이오팜 청약...,작년/NNG 말/NNB 대비/NNG 두/MM 배/NNG 가까이/MAG 늘/VV 어...,작년말 대비 두 배 가까이 늘어 지난달 26일 하룻새 4조 급증 SK바이오팜 청약...,"[작년, 대비, 지난달, 급증, 바이오팜, 청약, 뭉칫돈, 유입, 기관, 외국인, ..."
2,［중부매일 서인석 기자］ 이차영 괴산군수가 1일 열린 직원조회에서 민선 7기 2년...,［/SL 중부매/NNG 이/VCP ㄹ/ETM 서인석/NNP 기자/NNG ］/SL ...,중부매일 서인석 기자 이차영 괴산군수가 1일 열린 직원조회에서 민선 7기 2년간의...,"[중부, 서인석, 기자, 이차영, 괴산, 군수, 직원, 조회, 민선, 군정, 성과,..."
3,라임자산운용(라임)의 1조6000억원 펀드 환매 중단 사건을 수사 중인 검찰이 1...,라임자산운/NNG 용/XSN (/SS 라임/NNG )/SS 의/JKG 1/SN 조...,라임자산운용 라임 의 1조6000억원 펀드 환매 중단 사건을 수사 중인 검찰이 1...,"[라임, 자산, 운용, 라임, 펀드, 환매, 중단, 사건, 수사, 검찰, 신한은행,..."
4,시모네타 소마루가 스위스 대통령 [EPA=연합뉴스 자료사진] . (제네바=연합뉴스...,시모네/NNG 하/XSA 다/EC 소마루/NNG 가/JKS 스위스/NNP 대통령/...,시모네타 소마루가 스위스 대통령 EPA 연합뉴스 자료사진 제네바 연합뉴스 임은진 ...,"[시모네, 소마, 스위스, 대통령, 연합뉴스, 자료, 사진, 제네바, 연합뉴스, 임..."


In [51]:
#빈도 수 구하기.

# count_data = [y for x in df['nouns'].tolist() for y in x]

from collections import Counter
count = Counter(count_data)

node_df = pd.DataFrame(count.items(), columns=['node', 'nodesize'])
# node_df = node_df[node_df['nodesize'] >= 50]

# 형태소 데이터 / csv파일로 저장
df.to_csv('save/tmp_mecab_data2.csv', index=False)
# 카운트 데이터 / csv파일로 저장
node_df.to_csv('save/tmp_mecab_count2.csv', index=False)

node_df.head()

,node,nodesize
0,인천,2787
1,해양경찰,55
2,서장,266
3,신동,37
4,도서,166


In [5]:
from ast import literal_eval

# literal_eval이 기본 자료형만
# eval은 모든 스트링을 코드화

df = pd.read_csv('save/tmp_mecab_data2.csv',  converters={'nouns': literal_eval})
# df = pd.read_csv('save/tmp_data2.csv', converters={'nouns': eval})
node_df = pd.read_csv('save/tmp_mecab_count2.csv')

## 3. 연관 키워드 추출하기

### [트위터 연관 키워드 분석]

In [6]:
transactions = df['nouns'].tolist()
transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.and
# transactions

In [7]:
try:
    from mlxtend.preprocessing import TransactionEncoder
except:
    !pip install mlxtend
    from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import apriori as apriori2
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_result = te.fit(transactions).transform(transactions)

print(te_result.shape[0]*te_result.shape[1])

1112790970


In [8]:
df2 = pd.DataFrame(te_result, columns=te.columns_)
df2.head()

,가가,가가와,가가와현,가감,가갸,가건물,가게,가격,가격대,가격제,...,힐러리,힐리,힐링,힐스,힐책,힐튼,힘겨루기,힘줄,힘찬,힙합
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
# keyword = '손흥민'

df2 = pd.DataFrame(te_result, columns=te.columns_)

import time
start = time.time()  # 시작 시간 저장

# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
#------------------------------------
# 공통정보
#------------------------------------
# News Cnt = 21,194
# Mecab + 1글자제거 + 불용어제거 = 190초
# Transaction Cnt = 1,112,790,970 (11억)

#------------------------------------
# max_len=(default: None) 1:다 (무제한 관계)
#------------------------------------
#-- Apriori --
# 0.05 = 32초 / 1106개
# 0.03 = 251초 / 5974개
# 0.02 =  초 / 개 (메모리 부족.)
# 0.01 =  초 / 개 (메모리 부족.)

#-- Apriori=Low_memory --
# 0.05 = 41초

#-- fpgrowth --
# 0.05 = 10초
# 0.03 = 24.52초
# 0.01 = 383초

#-- fpgrowth=max_len(2) --
# 0.05 = 12초
# 0.03 = 27.58초
# 0.01 = 117초

#------------------------------------
# Apriori / max_len=2
#------------------------------------
# 0.05 = 22초 / 1,106개
# 0.03 = 72초 / 5,974개
# 0.01 = 1387초 / 75,812개

#------------------------------------
# Apriori / max_len=2 + low_memory=True (Low_memory는 1:1분석에만 효율이 있다 / fpgrowth는 1:다 분석에 효율이 있다.)
#------------------------------------
# 0.01 = 41초 / 75,812개 (세부분석 : 0.49초)
# 0.001 = 85초 / 1,895,362개 (세부분석 : 11초)
# 0.0009 = 84.7초 / 2,098,240개 (세부분석 : 11초)
# 0.0001 = 186초 / 15,440,440개 (세부분석 : 83초)

#------------------------------------
# fpgrowth / max_len=2 (교집합으로 제외시켜 나가는 구조)
#------------------------------------
# 0.05 = 12초
# 0.03 = 27.58초
# 0.01 = 117 초 / 75,812 개 (Apriori와 검색결과 같음)
# 0.001 = 
# 0.0009 = 오래걸려서 정지함.

#------------------------------------
# fpmax / max_len=2
#------------------------------------
# 0.01 = 106 초 / 41750 개  #분석 결과 다름 support_only=True


# itemset = apriori2(df2, min_support=0.05, use_colnames=True, max_len=2, low_memory=True)
itemset = apriori2(df2, min_support=0.01, use_colnames=True, low_memory=True)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

itemset.head()



KeyboardInterrupt: 

In [ ]:
start = time.time()  # 시작 시간 저장
#연결규칙
itemset2 = association_rules(itemset, metric="confidence", min_threshold=0.0000001)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
itemset2

In [31]:
# 연관키워드 알고리즘 데이터 csv저장
itemset2.to_csv('save/tmp_mecab_last2(0_0001).csv', index=False)

In [4]:
# load파일
itemset2 = pd.read_csv('save/tmp_mecab_last2(0_001).csv', converters={'antecedents': eval, 'consequents': eval})
itemset2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(관련),(가게),0.292488,0.003444,0.001038,0.003549,1.030365,0.000031,1.000105
1,(가게),(관련),0.003444,0.292488,0.001038,0.301370,1.030365,0.000031,1.012713
2,(기자),(가게),0.376050,0.003444,0.001038,0.002760,0.801409,-0.000257,0.999314
3,(가게),(기자),0.003444,0.376050,0.001038,0.301370,0.801409,-0.000257,0.893105
4,(때문),(가게),0.178541,0.003444,0.001510,0.008457,2.455212,0.000895,1.005055


In [15]:
# 조건이 키워드
keyword = '날씨'
# zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x))]

zz = itemset2[itemset2['antecedents'].apply(lambda x : (keyword in x) & (len(x) == 1))]
# zz = itemset2[itemset2['antecedents'].apply(lambda x : (len(x) == 1))]

zz = zz[zz['consequents'].apply(lambda x : (len(x) == 1))]

print(zz.shape)

#카운트 추가
node_df = pd.read_csv('save/tmp_mecab_count2.csv')
def getcount(data) :
    return node_df[node_df['node'].apply(lambda x : data == frozenset({x}))]['nodesize'].values[0]

def getcount_news(data) :
    return df2[next(iter(data))].sum()

# zz['출현_카운트'] = zz['consequents'].apply(lambda x : getcount(x))
# zz['뉴스_카운트'] = zz['consequents'].apply(lambda x : getcount_news(x))

# iat는 한개의 값만 사용할 수 있고 iloc나 loc는 슬라이싱 해서도 사용할 수 있다
# 슬라이싱 하면 결과가 문자값만 나오는게 아니라서 안예쁘지만

# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iat[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].iloc[0]
# node_df[node_df['node'].apply(lambda x : '의원' == x)]['nodesize'].values[0]



pd.set_option('display.max_rows', 1000)
qq = zz.sort_values(by=['support','confidence','lift'], axis='index', ascending=False)

qq.head(20)



(226, 9)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
614883,(날씨),(지역),0.012126,0.279607,0.005709,0.470817,1.683850,0.002319,1.361330
614775,(날씨),(오후),0.012126,0.174814,0.004813,0.396887,2.270345,0.002693,1.368212
614709,(날씨),(서울),0.012126,0.239455,0.003680,0.303502,1.267472,0.000777,1.091956
573791,(날씨),(기온),0.012126,0.009861,0.003492,0.287938,29.198816,0.003372,1.390523
574441,(날씨),(기자),0.012126,0.376050,0.003492,0.287938,0.765690,-0.001068,0.876258
614731,(날씨),(시작),0.012126,0.170001,0.003444,0.284047,1.670854,0.001383,1.159292
614921,(날씨),(코로나19),0.012126,0.331037,0.003397,0.280156,0.846297,-0.000617,0.929316
614693,(날씨),(사람),0.012126,0.165424,0.003303,0.272374,1.646516,0.001297,1.146984
614825,(날씨),(전국),0.012126,0.132207,0.003208,0.264591,2.001339,0.001605,1.180015
614667,(날씨),(발생),0.012126,0.181655,0.003208,0.264591,1.456559,0.001006,1.112776


In [29]:
#숫자가 제거됨.